In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src

In [ ]:
import os
import json
import operator
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from collections import Counter

from params import *
from utils.logger import Config, upload_to_kaggle

In [ ]:
from data.preparation import *
from data.processing import *
from data.post_processing import *

from utils.plot import *
from utils.metric import *
from inference.main import blend

## Experiment

In [ ]:
EXP_FOLDER = LOG_PATH + "2022-02-16/5/"  # 1. deberta-large - 0.8811
EXP_FOLDER = LOG_PATH + "2022-02-17/1/"  # 2. deberta-base - 0.8740
EXP_FOLDER = LOG_PATH + "2022-02-17/2/"  # 3. roberta-large - 0.8763
EXP_FOLDER = LOG_PATH + "2022-02-23/1/"  # 4. deberta-large-v3 - 0.8829
EXP_FOLDER = LOG_PATH + "2022-02-24/0/"  # 5. deberta-large - 0.8811
EXP_FOLDER = LOG_PATH + "2022-02-24/3/"  # 6. deberta-large-v3 - 0.8829
EXP_FOLDER = LOG_PATH + "2022-02-25/0/"  # 7.deberta-large-v3 - 0.8836
EXP_FOLDER = LOG_PATH + "2022-02-26/2/"  # 8. deberta-large-v3 MLM - 0.8808 

EXP_FOLDER = LOG_PATH + "2022-02-28/2/"  # 12. deberta-large-v3 pretrain pl1 - 0.8867
EXP_FOLDER = LOG_PATH + "2022-02-28/3/"  # 13. deberta-large-v3 pretrain pl1 - 0.8861
EXP_FOLDER = LOG_PATH + "2022-02-28/4/"  # 14. deberta-large-v3 pl1 1/1 - 0.8869
EXP_FOLDER = LOG_PATH + "2022-02-28/5/"  # 15. deberta-large-v3 pl1 1/1 - 0.8873
EXP_FOLDER = LOG_PATH + "2022-03-01/1/"  # 18. deberta-large-v3 pl1 1/1 - 0.8867
EXP_FOLDER = LOG_PATH + "2022-03-01/4/"  # 21. deberta-large-v3 pl1 all - 0.8890
EXP_FOLDER = LOG_PATH + "2022-03-02/0/"  # 24. deberta-large pl1 all - 0.8896
EXP_FOLDER = LOG_PATH + "2022-03-03/0/"  # 26. deberta-large-v3 pl1 5/1 - 0.8870
EXP_FOLDER = LOG_PATH + "2022-03-04/1/"  # 28. deberta-large-v3 pl1 10/1 bce upper - 0.8898

EXP_FOLDER = LOG_PATH + "2022-03-05/1/"  # 30. deberta-large-v3 pl2 None bce upper - 0.8907
EXP_FOLDER = LOG_PATH + "2022-03-06/2/"  # 32. deberta-large pl2 None - 0.8892
EXP_FOLDER = LOG_PATH + "2022-03-08/0/"  # 33. electra-large ce lower - 0.8894
EXP_FOLDER = LOG_PATH + "2022-03-22/0/"  # 34. deberta-large-v3 soft pl3 None bce - 0.8917
EXP_FOLDER = LOG_PATH + "2022-03-23/2/"  # 35. electra-large soft pl3 None bce - 0.8892
EXP_FOLDER = LOG_PATH + "2022-03-24/2/"  # 36. deberta-large soft pl3 None bce upper - 0.8911
EXP_FOLDER = LOG_PATH + "2022-03-24/3/"  # 37. deberta-v3(large soft pl3 10 mlm bce upper - 0.8911
EXP_FOLDER = LOG_PATH + "2022-03-22/0/"  # 38. deberta-large-v3 soft pl4 None bce upper - 0.8911
EXP_FOLDER = LOG_PATH + "2022-03-27/1/"  # 39. deberta-large-MLM soft pl4 None bce  - 0.8922
EXP_FOLDER = LOG_PATH + "2022-04-02/2/"  # 40. deberta-large-MLM2 soft pl4 None bce  - 0.8916
EXP_FOLDER = LOG_PATH + "2022-04-03/0/"  # 41. deberta-large-v3 soft pl4 None bce nodroput - 0.8917

EXP_FOLDER = LOG_PATH + "aphrodeep/02-26_3/" # 9. deberta-large-v3 - 0.8828
EXP_FOLDER = LOG_PATH + "aphrodeep/02-27_5/" # 11. electra-large - 0.8792
EXP_FOLDER = LOG_PATH + "aphrodeep/02-28_1/" # 16. electra-large pl1 1/1 - 0.8853
EXP_FOLDER = LOG_PATH + "aphrodeep/03-01_6/" # 19. electra-large pl1 1/1 - 0.8860
EXP_FOLDER = LOG_PATH + "aphrodeep/03-01_7/" # 20. deberta-large pl1 1/1 - 0.8845
EXP_FOLDER = LOG_PATH + "aphrodeep/03-02_0/" # 22. deberta-base-v3 pl1 1/1 - 0.8823
EXP_FOLDER = LOG_PATH + "aphrodeep/03-02_1/" # 23. roberta-large pl1 1/1 - 0.8834
EXP_FOLDER = LOG_PATH + "aphrodeep/03-02_2/" # 25. electra-large pl1 all - 0.8891
EXP_FOLDER = LOG_PATH + "aphrodeep/03-03_2/" # 27. roberta-large pl1 all - 0.8873
EXP_FOLDER = LOG_PATH + "aphrodeep/03-04_0/" # 29. electra-large pl1 all bce upper - 0.8896



EXP_FOLDER = LOG_PATH + "2022-04-04/0/"   # 43. v3-large - 0.8919
EXP_FOLDER = LOG_PATH + "2022-04-07/0/"   # 46. roberta-large - 0.8890
EXP_FOLDER = LOG_PATH + "2022-04-07/3/"   # 47. deberta-xlarge  soft pl4 x8 - 0.8921
EXP_FOLDER = LOG_PATH + "2022-04-08/0/"   # 48. deberta-v2-xlarge  soft pl4 x8 - 0.8917
EXP_FOLDER = LOG_PATH + "2022-04-08/5/"   # 49. PubMedBERT-base soft pl4 None 4ep- 0.8890
EXP_FOLDER = LOG_PATH + "2022-04-08/1/"   # 51. roberta-large - 0.889
EXP_FOLDER = LOG_PATH + "2022-04-09/0/"   # 52. deberta-base soft pl4 - 0.8910
EXP_FOLDER = LOG_PATH + "2022-04-09/1/"   # 53. roberta-base soft pl4 - 0.8890
EXP_FOLDER = LOG_PATH + "2022-04-10/0/"   # 54. deberta-v3-base soft pl3 - 0.891
EXP_FOLDER = LOG_PATH + "2022-04-10/1/"   # 55. roberta-base soft pl4 - 0.8890
EXP_FOLDER = LOG_PATH + "2022-04-10/2/"   # 56. PubMedBERT-base soft pl3 - 0.8890
EXP_FOLDER = LOG_PATH + "2022-04-10/3/"   # 57. roberta-large soft pl3 - 0.8908
EXP_FOLDER = LOG_PATH + "2022-04-11/1/"   # 58. deberta-v3-large soft pl3 - 0.8921
EXP_FOLDER = LOG_PATH + "2022-04-12/3/"   # 59. deberta-large soft pl3 - 0.8919

In [ ]:
EXP_FOLDERS = [  # 0.8897 -> PL1
    LOG_PATH + "2022-02-16/5/",  # 1. deberta-large - 0.8811
    LOG_PATH + "2022-02-25/0/",  # 7. deberta-large-v3 - 0.8836
    LOG_PATH + "2022-02-28/3/",  # 13. deberta-large-v3 pretrain pl1 - 0.8861
]

EXP_FOLDERS = [  # 0.8915 -> PL2
    LOG_PATH + "2022-03-01/1/",      # 18. deberta-large-v3 pl1 1/1 - 0.8867
    LOG_PATH + "aphrodeep/03-01_6/", # 19. electra-large 1/1- 0.8860
    LOG_PATH + "2022-03-02/0/",      # 24. deberta-large pl1 all - 0.8896
]

EXP_FOLDERS = [  # 0.8921 - LB 0.8889
    LOG_PATH + "2022-03-04/1/",      # 28. deberta-large-v3 extra diff 891 10/1 bce upper - 0.8898
    LOG_PATH + "2022-03-05/1/"       # 30. deberta-large-v3 pl2 None bce upper - 0.8907
]

EXP_FOLDERS = [  # 0.8919 - LB 0.8890 -> PL3
    LOG_PATH + "2022-03-02/0/",      # 24. deberta-large extra diff 891 all - 0.8896
    LOG_PATH + "2022-03-05/1/",      # 30. deberta-large-v3 pl2 None bce upper - 0.8907
]

EXP_FOLDERS = [  # 0.8930 - 0.891 -> PL4
    LOG_PATH + "2022-03-22/0/",  # 34. deberta-large-v3 soft pl3 None bce - 0.8917
    LOG_PATH + "2022-03-24/2/",  # 36. deberta-large soft pl3 None bce upper - 0.8911
    LOG_PATH + "2022-03-24/3/",  # 37. deberta-v3-large soft pl3 10 mlm bce upper - 0.8911
]

WEIGHTS = None

EXP_FOLDERS = [  # CV 0.8939 - LB 0.891
    LOG_PATH + "2022-03-22/0/",  # 34. deberta-large-v3 soft pl3 None - 0.8917
    LOG_PATH + "2022-04-07/3/",  # 47. deberta-xlarge  soft pl4 x8 - 0.8921

    LOG_PATH + "2022-04-09/0/",  # 52. deberta-base  soft pl4 - 0.8910
    LOG_PATH + "2022-04-09/1/",  # 53. roberta-base  soft pl3 - 0.8890EXP_FOLDER = LOG_PATH + "2022-04-10/2/"   # 57. PubMedBERT-base soft pl3 - 0.8890
    LOG_PATH + "2022-04-10/0/",  # 54. deberta-v3-base  soft pl3 - 0.8911
]

WEIGHTS = [1, 1, 0.5, 0.5, 0.5]


EXP_FOLDERS = [  # CV 0.8938 - LB 0.8919
    LOG_PATH + "2022-03-22/0/",   # 34. deberta-large-v3 soft pl3 None - 0.8917
    LOG_PATH + "2022-04-07/3/",   # 47. deberta-xlarge  soft pl4 x8 - 0.8921
    LOG_PATH + "2022-04-10/3/",   # 57. roberta-large soft pl3 - 0.8908
    LOG_PATH + "2022-04-10/0/",   # 54. deberta-v3-base soft pl3 - 0.891
]

WEIGHTS = [1, 1, 1, 1]

EXP_FOLDERS = [  # CV 0.8937 - LB 0.8919+ / 0.892 th=0.47
    LOG_PATH + "2022-04-07/3/",   # 47. deberta-xlarge  soft pl4 x8 - 0.8921
    LOG_PATH + "2022-04-10/3/",   # 57. roberta-large soft pl3 - 0.8908
    LOG_PATH + "2022-04-10/0/",   # 54. deberta-v3-base soft pl3 - 0.891
    LOG_PATH + "2022-04-11/1/",   # 58. deberta-v3-large soft pl3 - 0.8921
]

WEIGHTS = [1, 1, 1, 2]

# EXP_FOLDERS = [
#     LOG_PATH + "2022-04-10/0/",   # 54. deberta-v3-base soft pl3 - 0.891
#     LOG_PATH + "2022-04-11/1/",   # 58. deberta-v3-large soft pl3 - 0.8921
#     LOG_PATH + "2022-04-12/3/",   # 59. deberta-large soft pl3 - 0.8919
# ]

# WEIGHTS = [0.25, 1, 1]

In [ ]:
EXP_FOLDERS = [
    LOG_PATH + "2022-04-14/0/",   # 60. deberta-xlarge  soft pl3 x8 - 0.8911
#     LOG_PATH + "2022-04-13/1/",   # 60. deberta-xlarge  soft pl3 x8 - 0.8908
    
    LOG_PATH + "2022-04-10/0/",   # 54. deberta-v3-base soft pl3 - 0.891
    LOG_PATH + "2022-04-10/3/",   # 57. roberta-large soft pl3 - 0.8908
    LOG_PATH + "2022-04-11/1/",   # 58. deberta-v3-large soft pl3 - 0.8921
    LOG_PATH + "2022-04-12/3/",   # 59. deberta-large soft pl3 - 0.8919
]

WEIGHTS = [0, 0, 0, 1, 1]

In [ ]:
# EXP_FOLDERS = [
# #     LOG_PATH + "2022-04-22/1/",   # deberta-v3-large soft pl case 5µ
#     LOG_PATH + "2022-04-11/1/",   # 58. deberta-v3-large soft pl3 - 0.8921
# #     LOG_PATH + "2022-04-12/3/",   # 59. deberta-large soft pl3 - 0.8919
# ]
# WEIGHTS = None

In [ ]:
# upload_to_kaggle(EXP_FOLDERS, DATASETS_DIR + "weights_base/", "NBME Weights base")

## Load

In [ ]:
config = Config(json.load(open(EXP_FOLDERS[0] + "config.json", 'r')))

try:
    _ = config.lower
except AttributeError:
    config.lower = True

In [ ]:
df = load_and_prepare(root=DATA_PATH, lower=config.lower)

In [ ]:
# folder = LOG_PATH + "2022-04-10/0/"   # 54. deberta-v3-base soft pl3 - 0.891
# folder = LOG_PATH + "2022-04-11/1/"   # 58. deberta-v3-large soft pl3 - 0.8921

# pred_oof = np.load(folder + "pred_oof.npy", allow_pickle=True)
# from sklearn.metrics import *

# pred_cls = np.array([p.max() for p in pred_oof])
# tgt_cls = df['target'].apply(lambda x: int(np.max(x) > 0))

# ConfusionMatrixDisplay.from_predictions(tgt_cls, pred_cls > 0.5)
# plt.title(f'Accuracy={accuracy_score(tgt_cls, pred_cls > 0.5) :.4f}')
# plt.show()

In [ ]:
folds = pd.read_csv(config.folds_file)
df = df.merge(folds, how="left", on=["case_num", "pn_num"])

In [ ]:
pred_oof = blend(
    [np.load(f + "pred_oof.npy", allow_pickle=True).tolist() for f in EXP_FOLDERS],
    weights=WEIGHTS
)

# pred_oof = np.load('../output/pred_oof.npy', allow_pickle=True)


df["probs"] = pred_oof
# df["probs1"] = pred_oof
# pred_oof = np.load(EXP_FOLDERS[0] + "pred_oof.npy", allow_pickle=True)

In [ ]:
# from sklearn.metrics import *
# df = df[df['fold'] == 0].reset_index(drop=True)

# print(accuracy_score(
#     df['target'].apply(lambda x: np.max(x > 0.5)), 
#     df['probs'].apply(lambda x: np.max(x > 0.5)), 
# ))

# print(confusion_matrix(
#     preds_cls > 0.5,
#     df['target'].apply(lambda x: np.max(x > 0.5)),
# ))

# print(confusion_matrix(
#     df['probs'].apply(lambda x: np.max(x > 0.5)), 
#     preds_cls > 0.5,
# ))

In [ ]:
# preds_cls = np.load('../output/preds_cls_pl.npy')
# th = 1

# for i in range(len(preds_cls)):
#     if preds_cls[i] < th:
#         if df['probs'][i].max() > 0.5 and df['probs'][i].max() < 0.8:
#             print('erase')
#             df['probs'][i][:] = 0 
            
            

In [ ]:
# df_oof = pd.read_pickle("/home/theo/kaggle/nbme/output/oof_v15a.pkl")

# pred_oof_a = df_oof['char_probs'].values.tolist()
# pred_oof_b = df_oof['char_probs_b'].values.tolist()

# pred_oof = []
# for a, b in zip(pred_oof_a, pred_oof_b):
#     pred_oof.append(np.array([
#         1 - a,
#         a - b,
#         b,
#     ]).T)

In [ ]:
# EXP_FOLDERS = [
#     LOG_PATH + "2022-04-22/1/",   # deberta-v3-large soft pl case 5
# #     LOG_PATH + "2022-04-11/1/",   # 58. deberta-v3-large soft pl3 - 0.8921
# #     LOG_PATH + "2022-04-12/3/",   # 59. deberta-large soft pl3 - 0.8919
# ]
# WEIGHTS = None

# pred_oof = blend(
#     [np.load(f + "pred_oof.npy", allow_pickle=True).tolist() for f in EXP_FOLDERS],
#     weights=WEIGHTS
# )

# # df["probs"] = pred_oof
# df["probs1"] = pred_oof
# # pred_oof = np.load(EXP_FOLDERS[0] + "pred_oof.npy", allow_pickle=True)

In [ ]:
# def specific_case(p1, p2, case):
#     if case in [3, 5]:
#         return preds_to_labels([p2])[0]
#     else:
#         return preds_to_labels([p1])[0]
    
# df["preds"] = df.apply(lambda x: specific_case(x.probs, x.probs1, x.case_num), 1)

In [ ]:
# if pred_oof[0].shape[-1] == 3:
#     pred_oof = [p[:, 1:].sum(-1) for p in pred_oof]

In [ ]:
PROB_THRESHOLD = 0.5


# PROB_THRESHOLDS = [0.4, 0.6, 0.4, 0.6, 0.8, 0.5, 0.6, 0.6, 0.5, 0.5]
# PROB_THRESHOLDS = [0.34, 0.33, 0.45, 0.6, 0.79, 0.47, 0.55, 0.61, 0.75, 0.3]
# PROB_THRESHOLDS = [0.4, 0.4, 0.5, 0.5, 0.6, 0.5, 0.5, 0.5, 0.6, 0.4]

In [ ]:
# for i in range(len(df)):
#     if df['probs'][i].max() > 0.5 and df['probs'][i].max() < 0.8:
# #         print('erase')
#         df['probs'][i][:] = 0 

In [ ]:
# df['probs'] = pred_oof
df['preds'] = preds_to_labels(df["probs"].values.tolist(), threshold=PROB_THRESHOLD)
# df['preds'] = preds_to_labels_advanced(pred_oof, df['case_num'].values, thresholds=PROB_THRESHOLDS)

df['preds_pp'] = df.apply(lambda x: post_process_spaces(x['preds'], x['clean_text']), 1)

In [ ]:
for i in range(5):
    df_ = df[df['fold'] == i].reset_index()
    cv_f1 = micro_f1(df_['preds_pp'], df_['target'])
    print(f"-> Fold #{i} score : {cv_f1:.4f}")
#     break

cv_f1 = micro_f1(df['preds_pp'], df['target'])
print(f"\n-> CV score: {cv_f1:.4f}")

## Blacklist

In [ ]:
delta = 0
preds_pp_blacklist = []
preds_pp = []
gts = []

for fold in range(5):
    for case in range(10):
        dfc = df[df["case_num"] == case]
        for ft in tqdm(dfc['feature_num'].unique()):
            dfcft = dfc[dfc["feature_num"] == ft].reset_index(drop=True)
            df_train = dfcft[dfcft["fold"] != fold].reset_index(drop=True)
            df_val = dfcft[dfcft["fold"] == fold].reset_index(drop=True)

    #         print(f'\n -> Feature {dfcft["feature_text"][0]}')

            tps_train, fps_train, fns_train, tns_train = [], [], [], []

            for i, (text, feature_text, pred, tgt) in enumerate(
                df_train[["pn_history", "feature_text", "preds_pp", "target"]].values,  start=1
            ):
                tps, fps, fns = np.zeros(len(text)), np.zeros(len(text)), np.zeros(len(text))

                for j in range(len(text)):
                    if pred[j] and tgt[j]:
                        tps[j] = 1
                    elif pred[j] and not tgt[j]:
                        fps[j] = 1
                    elif not pred[j] and tgt[j]:
                        fns[j] = 1

                tns = char_target_to_span(1 - np.max([tps, fps, fns], 0))
                tps = char_target_to_span(tps)
                fps = char_target_to_span(fps)
                fns = char_target_to_span(fns)

                for span in tps:
                    tps_train.append(text[span[0]: span[1]].strip().lower())
                for span in fps:
                    fps_train.append(text[span[0]: span[1]].strip().lower())
                for span in fns:
                    fns_train.append(text[span[0]: span[1]].strip().lower())
                for span in tns:
                    tns_train.append(text[span[0]: span[1]].strip().lower())

#             count = Counter(fps_train)
            to_remove = [i for i in fps_train if i not in tps_train] # and count[i] > 1]

            for i, (text, feature_text, pred, tgt) in enumerate(
                df_val[["pn_history", "feature_text", "preds_pp", "target"]].values,  start=1
            ):
                new_pred = pred.copy()
                spans = char_target_to_span(pred)

                for span in spans:
                    if text[span[0]: span[1]].strip().lower() in to_remove:
                        new_pred[span[0]: span[1]] = 0
                        p = tgt[span[0]: span[1]]
                        print(f'Removed "{text[span[0]: span[1]].strip().lower()}" - {p}')
                        delta += (p == 0).sum()
                        delta -= (p > 0).sum()
                    
                preds_pp_blacklist.append(new_pred)
                preds_pp.append(pred)
                gts.append(tgt)


In [ ]:
delta

In [ ]:
micro_f1(preds_pp, gts)

In [ ]:
micro_f1(preds_pp_blacklist, gts)

### Tweak threshold

In [ ]:
# scores = {}
# # for threshold in np.arange(0.4, 0.75, 0.05):
# for threshold in np.arange(0.47, 0.511, 0.01):
#     df['preds'] = preds_to_labels(pred_oof, threshold=threshold)
#     df['preds_pp'] = df.apply(lambda x: post_process_spaces(x['preds'], x['clean_text']), 1)

#     cv_f1 = micro_f1(df['preds_pp'], df['target'])
#     scores[threshold] = cv_f1

#     print(f"Threshold : {threshold:.2f}\t-  CV score : {cv_f1:.4f}")

# PROB_THRESHOLD = max(scores, key=scores.get)

# print(f'\n -> Best score {max(scores.values()) :.4f} with threshold {PROB_THRESHOLD:.2f}')

# PROB_THRESHOLD = 0.5

In [ ]:
case_scores = {}

for case in sorted(df['case_num'].unique()):
    ths = []
    for fold in range(5):
        scores = {}
        for threshold in np.arange(0.3, 0.85, 0.05):
            df['preds'] = preds_to_labels(pred_oof, threshold=threshold)

            dfc = df[df['case_num'] == case].reset_index()
            dfc = dfc[dfc['fold'] == fold].reset_index()
            dfc['preds_pp'] = dfc.apply(lambda x: post_process_spaces(x['preds'], x['clean_text']), 1)

            cv_f1 = micro_f1(dfc['preds_pp'], dfc['target'])
            scores[np.round(threshold, 2)] = cv_f1

        PROB_THRESHOLD = max(scores, key=scores.get)
        print(f'-> Case {case} - Best score {max(scores.values()) :.4f} with threshold {PROB_THRESHOLD:.2f}')
        ths.append(PROB_THRESHOLD)

        score = cv_f1 = micro_f1(dfc['preds_pp'], dfc['target'])
        case_scores[case] = scores
    print(f"Avg : {np.mean(ths):.2f} \n")

In [ ]:
# best_scores = []
# ref_scores = []
# thresholds = []

# plt.figure(figsize=(15, 20))
# for i in range(len(case_scores)):
#     plt.subplot(4, 3, i + 1)
#     plt.grid(True)
    
#     plt.plot(case_scores[i].keys(), case_scores[i].values())
    
#     th = max(case_scores[i], key=case_scores[i].get)
# #     th = np.round(th * 10) / 10
#     thresholds.append(th)
#     s = case_scores[i][th]
#     ref = case_scores[i][0.5]
#     plt.title(f'Case {i} - Best {s :.4f} th={th:.2f} (+ {s - ref:.4f})')
    
#     best_scores.append(s)
#     ref_scores.append(ref)

# print(f'Best score : {np.mean(best_scores) :.4f}')
# print(f'thresholds = {thresholds}')
# print(f'Ref score : {np.mean(ref_scores) :.4f}')
# plt.show()

## Preds viz

In [ ]:
df['target'].apply(lambda x: np.sum(x > 0)).sum(), df['preds_pp'].apply(lambda x: np.sum(x > 0.5)).sum()

In [ ]:
df['annotated'] = df['target'].apply(lambda x: np.max(x) > 0.5)
df['found'] = df['preds_pp'].apply(lambda x: np.max(x) > 0.5)

In [ ]:
df_fps = df[~df['annotated'] & df['found']]
len(df_fps)

In [ ]:
df_fns = df[df['annotated'] & ~df['found']]
len(df_fns)

In [ ]:
df_tps = df[df['annotated'] & df['found']]
len(df_tps)

In [ ]:
df_tns = df[~df['annotated'] & ~df['found']]
len(df_tns)

In [ ]:
# for c in range(10):
#     print(c)
#     dfg = df[df['case_num'] == c][['annotated', 'found', 'feature_text']].groupby('feature_text').mean()

#     display(dfg)

In [ ]:
dfg = df[['case_num', 'pn_num', 'preds_pp', 'target']].groupby(['case_num', 'pn_num']).agg(list).reset_index()
dfg['score'] = dfg.apply(lambda x: micro_f1(x.preds_pp, x.target), 1)

In [ ]:
c = 3
dfc = dfg[dfg['case_num'] == c].reset_index(drop=True)
dfc

In [ ]:
plt.figure(figsize=(15, 8))

for c in range(10):
    dfc = dfg[dfg['case_num'] == c].reset_index(drop=True)
    plt.scatter(
#         dfc['pn_num'] % 10000, 
        [i for i in range(len(dfc))],
        dfc['score'], 
        marker="x", 
        label=c
    )

plt.legend()    
plt.show()

In [ ]:
# df = df[df['location'].apply(lambda x: any([';' in y for y in x]))].reset_index(drop=True)

In [ ]:
len(df)

In [ ]:
df = df[df['pn_history'].apply(lambda x: "  " in x)].reset_index()

In [ ]:
df['pn_history'][0]

In [ ]:
len(df)

In [ ]:
case = 5

for k, i in enumerate(df[df["case_num"] == case]['pn_num'].unique()):
    plot_predictions(df, i, pred_col="preds_pp", max_score=1)

#     if k > 100:
#         break

### Score thresh PP

In [ ]:
scores = {}
for threshold in np.arange(PROB_THRESHOLD, PROB_THRESHOLD + 0.25, 0.01):
    df['preds_pp_score'] = df.apply(
        lambda x: post_process_score(x['probs'], x['preds_pp'], threshold=threshold), 1
    )

#     df['preds_pp_score'] = df.apply(
#         lambda x: post_process_score_grouped(
#             x['probs'], x['preds_pp'], proba_threshold=PROB_THRESHOLD, score_threshold=threshold
#         ), 1
#     )
#     df['preds_pp_score'] = df.apply(
#         lambda x: post_process_spaces(x['preds_pp_score'], x['clean_text']), 1
#     )

    cv_f1 = micro_f1(df['preds_pp_score'], df['target'])
    scores[threshold] = cv_f1

    print(f"Threshold : {threshold:.2f}\t-  CV score PP: {cv_f1:.4f}")

SCORE_THRESHOLD = max(scores, key=scores.get)

print(f'\n -> Best score {max(scores.values()) :.4f} with threshold {SCORE_THRESHOLD:.2f}')

In [ ]:
df['preds_pp_score'] = df.apply(
    lambda x: post_process_score(x['probs'], x['preds_pp'], threshold=SCORE_THRESHOLD), 1
)

In [ ]:
df_diff = df[df.apply(lambda x: (x['preds_pp'] != x['preds_pp_score']).any(), 1)].reset_index()


for k, i in enumerate(df_diff['pn_num'].unique()):
    plot_predictions(df_diff, i, pred_col="preds_pp")
    print('\n')

    plot_predictions(df_diff, i, pred_col="preds_pp_score")
    print('\n')
    
        
    if k > 1:
        break

### Words

In [ ]:
import nltk

def resplit_words(words):
    new_words = []
    for w in words:
        for c in ["/", "-", ".", "\\", ':', ',', '+', '=']:
            w = f" {c} ".join(w.split(c))
            w = f" {c} ".join(w.split(c))
        new_words += filter(None, w.split(' '))
        
    return new_words

def char_to_words(text, target):
    words = nltk.word_tokenize(text)
    words = resplit_words(words)
    
    word_targets = []
    current_idx = 0
    
    for word in words:
        if word in ['``', "''"]:
            word = '"'
            
        start_idx = text.find(word, current_idx)

        if start_idx >= 0:
            word_targets.append(target[start_idx: start_idx + len(word)] )
            current_idx = start_idx + len(word)
        else:
            print(word)
        
    return words, word_targets

In [ ]:
idx = 0

words, word_targets = char_to_words(df['clean_text'][idx], df['target'][idx])

In [ ]:

    
for idx in tqdm(range(len(df))):
    
    words, word_targets = char_to_words(df['clean_text'][idx], df['target'][idx])
    
#     print(words)
    
    for w, t in zip(words, word_targets):
        if not len(np.unique(t > 0)) == 1:
            print(idx, w, t)
            
#             break
#     break

In [ ]:
text

In [ ]:
text = df['clean_text'][0]
words = nltk.word_tokenize(text)

In [ ]:
text.find(words[0], current_idx)

## Af-idf

In [ ]:
df_ft = df[df['feature_num'] == 0].reset_index()

In [ ]:
for k, i in enumerate(df_ft['pn_num'].unique()):
    plot_predictions(df_ft, i, pred_col="preds_pp", max_score=1)

In [ ]:
import nltk

def split_to_words(chars):
    for c in ['?', '!', '-', '/', ',', '.']:
        chars = chars.replace(c, f" {c} ")
    return nltk.word_tokenize(chars)
#     return chars.split(' ')

In [ ]:
import operator

def af_idf(df):

    annotated_words, words = [], []
    for i in range(len(df)):
        text = df['clean_text'][i].lower()
        annotation = " ".join(df['annotation'][i]).lower()

        words += split_to_words(text)
        if len(annotation):
            annotated_words += split_to_words(annotation)   

    af_idf_dic = dict(Counter(annotated_words))
    word_count = Counter(words)
    
    for k in af_idf_dic.keys():
        try:
            af_idf_dic[k] /= word_count[k]
        except:
            af_idf_dic[k] = 1
    
    return dict(sorted(af_idf_dic.items(), key=lambda x: x[1], reverse=True))

In [ ]:
ft = df_ft['feature_text'][0]
idf = af_idf(df_ft)

In [ ]:
ft

In [ ]:
idf

### Len PP
- Doesn't work unless you overfit

## Preds viz

In [ ]:
for k, i in enumerate(df['pn_num'].unique()):
    plot_predictions(df, i, pred_col="preds_pp_score")
    print('\n')
    
#     plot_predictions(df, i, pred_col="preds_pp_inc")
#     print('\n')
        
    if k > 1:
        break

#### Split

In [ ]:
col = "preds_pp_score"
col_group = "feature_num"

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=5000)
splits = list(skf.split(df, y=df[col_group]))


split = 0
stats_idx = splits[split][0]
select_idx = splits[split][0]
val_idx = splits[split][1]

# stats_idx, select_idx, val_idx = None, None, None

#### Stats

In [ ]:
df_stats = df.iloc[stats_idx].reset_index(drop=True) if stats_idx is not None else df.copy()

df_stats['annot_lens'] = df_stats['annotation'].apply(lambda x: [len(y) for y in x])
# df_stats['annot_lens'] = df_stats['annot_lens'].apply(lambda x: np.min(y) if len(y) else 0)

min_lens = df_stats[["feature_num", "annot_lens"]].groupby('feature_num').agg(
    lambda x: np.min(np.concatenate(list(x))) if len(np.concatenate(list(x))) else 0
).reset_index()

df_stats = df_stats.drop('annot_lens', axis=1)

#### Find features to PP

In [ ]:
def post_process_len(text, pred, min_len=0, delta=2):
    if pred.sum() < min_len - delta:
        pred = np.zeros(len(pred))

    return pred

In [ ]:
def micro_f1_grouped(df, col_group, col_pred="pred", col_truth="target"):
    dfg = df[[col_group, col_pred, col_truth]].groupby(col_group).agg(list)

    dfg['score'] = dfg.apply(lambda x: micro_f1(x[col_pred], x[col_truth]), 1)

    return dfg['score'].reset_index()

In [ ]:
df_train = df.iloc[select_idx].reset_index(drop=True) if select_idx is not None else df.copy()
df_train = df_train.merge(min_lens, on="feature_num")

In [ ]:
df_train['preds_pp_len'] = df_train.apply(
    lambda x: post_process_len(x['clean_text'], x['preds_pp_score'], x['annot_lens']), 1
)

In [ ]:
dfg = micro_f1_grouped(df_train, col_group, col)
scores_ref = dfg['score'].values

In [ ]:
dfg = micro_f1_grouped(df_train, col_group, "preds_pp_len")
scores_pp = dfg['score'].values

In [ ]:
min_gap = 0.001
to_replace = []

for i in range(len(scores_ref)):
    if scores_pp[i] > 0 and scores_ref[i] > 0:
        gap = scores_pp[i] - scores_ref[i]
        if gap > min_gap: # or gap < -0.001:
            print(f'{col_group} {dfg[col_group][i]:03d} : {gap:.3f}')

        if gap > min_gap:
            to_replace.append(dfg[col_group][i])

### Val 

In [ ]:
df_val = df.iloc[val_idx].reset_index(drop=True) if select_idx is not None else df_.copy()
df_val = df_val.merge(min_lens, on="feature_num")

In [ ]:
df_val['preds_pp_len'] = df_val.apply(
    lambda x: post_process_len(x['clean_text'], x[col], x['annot_lens']), 1
)

In [ ]:
df_val['pred_pp_len_opti'] = df_val.apply(
    lambda x: x['preds_pp_len'] if x[col_group] in to_replace else x[col], 1
)

In [ ]:
cv_f1 = micro_f1(df_val[col], df_val['target'])

print(f"-> Val score PP: {cv_f1:.4f}")

In [ ]:
cv_f1 = micro_f1(df_val['preds_pp_len'], df_val['target'])

print(f"-> Val score PP: {cv_f1:.4f}")

In [ ]:
cv_f1 = micro_f1(df_val['pred_pp_len_opti'], df_val['target'])

print(f"-> Val score PP: {cv_f1:.4f}")

In [ ]:
df_diff = df_val[df_val.apply(lambda x: (x['pred_pp_len_opti'] != x['preds_pp_score']).any(), 1)].reset_index()

for k, i in enumerate(df_diff['pn_num'].unique()):
    plot_predictions(df_diff, i, pred_col="preds_pp_score")
    print('\n')

    plot_predictions(df_diff, i, pred_col="pred_pp_len_opti")
    print('\n')
        
#     if k > 1:
#         break

### Test adj pp

In [ ]:
def post_process_adj(preds):
    preds = np.copy(preds)
    for i in range(1, len(preds) - 1):
        if preds[i]:
            continue
        if preds[i + 1] and preds[i - 1]:
            preds[i] = 1
    return preds

In [ ]:
# df['preds_pp_ajd'] = df['preds_pp'].apply(post_process_adj)

In [ ]:
# cv_f1 = micro_f1(df['preds_pp_ajd'].values.tolist(), df['target'].values.tolist())

# print(f"-> CV score PP: {cv_f1:.5f}")

In [ ]:
def is_pos(target, idx, error="none"):
    if target[idx] == "":
        if error == "left":
            return is_pos(target, idx - 1, error=error)
        elif error == "right": 
            return is_pos(target, idx + 1, error=error) 
        else:
            return False
    return int(target[idx]) > 0

In [ ]:
def post_process_ajd_w(preds, text):
    preds = preds.astype(str)

    for i, char in enumerate(text):
        if char == " ":
            preds[i] = " "
            
    target = "".join(preds)
    
    target_words = target.split(' ')
    
#     for i in range(1, len(target_words) - 1):
# #         if not len(target_words[i]):
# #             continue

#         if is_pos(target_words, i):
#             continue

#         if is_pos(target_words, i - 1) and is_pos(target_words, i + 1):
#             target_words[i] = "1" * len(target_words[i])

    target = "0".join(target_words)
    target = np.array(list(target)).astype(int)
    
    target = post_process_spaces(target, text)

    return target

In [ ]:
df['preds_pp_adj_w'] = df.apply(lambda x: post_process_ajd_w(x['preds'], x['clean_text']), 1)

In [ ]:
cv_f1 = micro_f1(df['preds_pp_adj_w'], df['target'])

print(f"-> CV score PP: {cv_f1:.5f}")

### PP Incomplete words
- Doesn't work

In [ ]:
df['preds_pp_inc'] = df.apply(lambda x: post_process_incomplete(x['preds_pp_score'], x['pn_history']), 1)
df['preds_pp_inc'] = df.apply(lambda x: post_process_spaces(x['preds_pp_inc'], x['clean_text']), 1)

In [ ]:
cv_f1 = micro_f1(df['preds_pp_inc'].values.tolist(), df['target'].values.tolist())

print(f"-> CV score PP: {cv_f1:.4f}")

In [ ]:
df_diff = df[df.apply(lambda x: (x['preds_pp_score'] != x['preds_pp_inc']).any(), 1)]

In [ ]:
# for k, i in enumerate(df_diff['pn_num'].unique()):
#     plot_predictions(df_diff, i, pred_col="preds_pp")
#     print('\n')
    
#     plot_predictions(df_diff, i, pred_col="preds_pp_inc")
#     print('\n')
        
# #     if k > 1:
# #         break